# Post Here: Subreddit Predictor

## Recommendation API - 1.2

> aka: the Most Voluptuous Pipeline

---
---

## Intro - MVP Classifier (Model #3)

This is the third iteration of the model for predicting the most appropriate subreddits for a given post.
This iteration will use a pipeline containing the vectorizer, labelencoder, and model.

The model will be trained using data from the [reddit self-post classification task dataset](https://www.kaggle.com/mswarbrickjones/reddit-selfposts), available on Kaggle thanks to [Evolution AI](https://evolution.ai//blog/page/5/an-imagenet-like-text-classification-task-based-on-reddit-posts/).

---

### Load and preprocess the data

In [1]:
# === General imports === #
import pandas as pd
import numpy as np
import os

In [13]:
# === sklearn imports === #
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

In [4]:
# === Load the dataset === #
data_filepath = "../assets/rspct_10k.csv"

# Tab-separated, 10k rows
df1 = pd.read_csv(data_filepath, sep="\t")

df1.shape

(10000, 4)

In [5]:
# === Split up dataset into train and test === #
# from sklearn.model_selection import train_test_split

# 80% train, 20% test, stratified on the target
train, test = train_test_split(df1, test_size=0.2, stratify=df1["subreddit"])

train.shape, test.shape

((8000, 4), (2000, 4))

In [6]:
# === Arrange data into feature and target === #

# MVP model only uses 'selftext' feature
X_train = train["selftext"]
X_test = test["selftext"]

# Predict the subreddit of each post
y_train = train["subreddit"]
y_test = test["subreddit"]

print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

(8000,) (2000,)
(8000,) (2000,)


---
---

### Pipeline

In [11]:
# === Instantiate encoder + vectorizer + classifier === #

# from sklearn.preprocessing import LabelEncoder
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.naive_bayes import MultinomialNB

le = LabelEncoder()
tfidf = TfidfVectorizer(stop_words="english")
nb = MultinomialNB(alpha=0.1)

In [12]:
# === Define the pipeline === #
pipe = Pipeline([
                 ("le", le),  # Encoder
                 ("tfidf", tfidf),  # Vectorizer
                 ("nb", nb),  # Classifier
                ])

In [17]:
# === Define the parameter grid === #
parameters = {
    "tfidf__max_df": (5, 8),
    "tfidf__max_features": (20000, 30000),
    "nb__alpha": (0.1, 0.3),
}

In [18]:
# === Run grid search === #
# from sklearn.model_selection import GridSearchCV

grid_search = GridSearchCV(pipe, parameters, cv=3, n_jobs=-1, verbose=1)
grid_search.fit(X_train, y_train)

Fitting 3 folds for each of 8 candidates, totalling 24 fits


/Users/Tobias/.vega/neural_draft-sDaGcGcX/lib/python3.7/site-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=3.
  % (min_groups, self.n_splits)), UserWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  24 out of  24 | elapsed:    3.0s finished


TypeError: fit_transform() takes 2 positional arguments but 3 were given

In [8]:
# Transform both using the train fit
y_train = le.transform(y_train)
y_test  = le.transform(y_test)

y_train[:8]

array([691, 289, 117, 952,  77, 532, 332, 995])

In [25]:
# === Vectorize! === #

# Extract features from the text data using bag-of-words (single words + bigrams).
# Uses tfidf weighting (helps a little for Naive Bayes in general).


# Fit the vectorizer on the feature column to create vocab (doc-term matrix)
# This process is split into component parts (fit / transform) for pickling
vocab = tfidf.fit(X_train)

# Get sparse document-term matrices
X_train_sparse = vocab.transform(X_train)
X_test_sparse = vocab.transform(X_test)

In [26]:
# === Naive Bayes model === #


# Instantiate and train the model

nb.fit(X_train_sparse, y_train)

MultinomialNB(alpha=0.1, class_prior=None, fit_prior=True)

In [27]:
# === Create predictions on test feature === #
y_pred_proba = nb.predict_proba(X_test_sparse)

print(y_pred_proba.shape)

(2000, 1013)


In [28]:
# === For each prediction, find the index with the highest probability === #
# import numpy as np
y_pred = np.argmax(y_pred_proba, axis=1)
y_pred[:10]

array([279, 279, 279, 279, 598, 560, 185, 553, 185, 185])

In [29]:
# === Evaluate performance using precision-at-k === #

def precision_at_k(y_true, y_pred, k=5):
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    y_pred = np.argsort(y_pred, axis=1)
    y_pred = y_pred[:, ::-1][:, :k]
    arr = [y in s for y, s in zip(y_true, y_pred)]
    return np.mean(arr)

print('precision@1 =', np.mean(y_test == y_pred))
print('precision@3 =', precision_at_k(y_test, y_pred_proba, 3))
print('precision@5 =', precision_at_k(y_test, y_pred_proba, 5))

# 1,000,000 records
# precision@1 = 0.6732724580454097
# precision@3 = 0.8058588351431392
# precision@5 = 0.8481391905231984

# 100,000 records
# precision@1 = 0.45935
# precision@3 = 0.6075
# precision@5 = 0.665

# 10,000 records
# precision@1 = 0.108
# precision@3 = 0.1785
# precision@5 = 0.215

precision@1 = 0.108
precision@3 = 0.1785
precision@5 = 0.215


---

### Generate Predictions from new input

In [18]:
# === Example post === #

# The example comes from 'r/learnprogramming'
post = """I am a new grad looking for a job and currently in the process with a company for a junior backend engineer role. I was under the impression that the position was Javascript but instead it is actually Java. My general programming and "leet code" skills are pretty good, but my understanding of Java is pretty shallow. How can I use the next three days to best improve my general Java knowledge? Most resources on the web seem to be targeting complete beginners. Maybe a book I can skim through in the next few days?

Edit:

A lot of people are saying "the company is a sinking ship don't even go to the interview". I just want to add that the position was always for a "junior backend engineer". This company uses multiple languages and the recruiter just told me the incorrect language for the specific team I'm interviewing for. I'm sure they're mainly interested in seeing my understanding of good backend principles and software design, it's not a senior lead Java position."""

In [23]:
# === Function to serve predictions === #
# The main functionality of the predict API endpoint

def predict(post: str, n: int = 5) -> dict:
    """
    Serve subreddit predictions.
    
    Parameters
    ----------
    post : string
        Selftext that needs a home.
    n    : integer
        The desired name of the output file,
        not including the '.pkl' extension.

    Returns
    -------
    Python dictionary formatted as follows:
        [{'subreddit': 'PLC', 'proba': 0.014454},
         ...
         {'subreddit': 'Rowing', 'proba': 0.005206}]
    """
    
    # Vectorize the post -> sparse doc-term matrix
    post_vec = vocab.transform([post])
    
    # Generate predicted probabilities from trained model
    proba = nb.predict_proba(post_vec)
    
    # Wrangle into correct format
    return (pd
                .DataFrame(proba, columns=[le.classes_])  # Classes as column names
                .T  # Transpose so column names become index
                .reset_index()  # Pull out index into a column
                .rename(columns={"level_0": "subreddit", 0: "proba"})  # Rename for aesthetics
                .sort_values(by="proba", ascending=False)  # Sort by probability
                .iloc[:n]  # n-top predictions to serve
                .to_dict(orient="records")
               )

In [24]:
# === Test out the function === #
post_pred = predict(post)  # Default is 5 results
post_pred

[{'subreddit': 'PLC', 'proba': 0.014454003455867464},
 {'subreddit': 'sales', 'proba': 0.008377114822879663},
 {'subreddit': 'AskHR', 'proba': 0.007373675677628392},
 {'subreddit': 'OccupationalTherapy', 'proba': 0.00561548847793111},
 {'subreddit': 'Rowing', 'proba': 0.005206212277479405}]

In [30]:
# === Test it out with another dummy post === #

# This one comes from r/suggestmeabook
post2 = """I've been dreaming about writing my own stort story for a while but I want to give it an unexpected ending. I've read lots of books, but none of them had the plot twist I want. I want to read books with the best plot twists, so that I can analyze what makes a good plot twist and write my own story based on that points. I don't like romance novels and I mostly enjoy sci-fi or historical books but anything beside romance novels would work for me, it doesn't have to be my type of novel. I'm open to experience after all. I need your help guys. Thanks in advance."""

In [31]:
# === This time with 10 results === #
post2_pred = predict(post2, n=10)
post2_pred

[{'subreddit': 'PLC', 'proba': 0.008913827946281429},
 {'subreddit': 'JUSTNOMIL', 'proba': 0.005364412871980992},
 {'subreddit': 'vaginismus', 'proba': 0.005343515334435943},
 {'subreddit': 'Rowing', 'proba': 0.004869320257487137},
 {'subreddit': 'hookah', 'proba': 0.00434668954125044},
 {'subreddit': 'dresdenfiles', 'proba': 0.004295750109887567},
 {'subreddit': 'StudentLoans', 'proba': 0.004077573292318194},
 {'subreddit': 'productivity', 'proba': 0.004050498345338448},
 {'subreddit': 'flexibility', 'proba': 0.003925624264099203},
 {'subreddit': 'dpdr', 'proba': 0.003923252551916036}]

---

### Picklization

In [61]:
# === Create pickle func to make pickling (a little) easier === #

def picklizer(to_pickle, filename, path):
    """
    Creates a pickle file.
    
    Parameters
    ----------
    to_pickle : Python object
        The trained / fitted instance of the 
        transformer or model to be pickled.
    filename : string
        The desired name of the output file,
        not including the '.pkl' extension.
    path : string or path-like object
        The path to the desired output directory.
    """
    import os
    import pickle

    # Create the path to save location
    picklepath = os.path.join(path, filename)

    # Use context manager to open file
    with open(picklepath, "wb") as p:
        pickle.dump(to_pickle, p)

In [62]:
# === Picklize! === #
filepath = "../assets"

# Export pipeline as pickle
picklizer(vocab, "03_pipe.pkl", filepath)